Action1	"Santa的接待安排
圣诞节前100天，Santa开放了workshop，欢迎以家庭单位的参观，如何更合理的安排这些家庭参观？
每个家庭有10个选择choice0-9，数字代表了距离圣诞节的天数，比如 1代表12月24日，每个家庭必须并且只安排一次参观
家庭数量 5000，即family_id 为[0, 4999]，每天访问的人数需要在125-300人
为了更合理的计算Santa的安排能力，我们使用preference cost和accounting penalty两个指标
1）preference cost，代表Santa的个性化安排能力
2）accounting penalty，代表Santa安排的财务成本
每天接待的人员数N(d)如果大于125，就会拥挤，产生过多的清洁成本
最终的 Score = preference cost + accounting penalty
最终提交每个家庭的安排 submission.csv
"

In [1]:
import pandas as pd

In [2]:
import numpy as np

In [3]:
data = pd.read_csv("family_data.csv",index_col='family_id')

In [4]:
data

,choice_0,choice_1,choice_2,choice_3,choice_4,choice_5,choice_6,choice_7,choice_8,choice_9,n_people
family_id,,,,,,,,,,,
0,52,38,12,82,33,75,64,76,10,28,4
1,26,4,82,5,11,47,38,6,66,61,4
2,100,54,25,12,27,82,10,89,80,33,3
3,2,95,1,96,32,6,40,31,9,59,2
4,53,1,47,93,26,3,46,16,42,39,4
...,...,...,...,...,...,...,...,...,...,...,...
4995,16,1,66,33,18,70,56,46,86,60,4
4996,88,66,20,17,26,54,81,91,59,48,2
4997,32,66,54,17,27,21,74,81,3,7,6


In [5]:
# n代表家庭成员个数，如果满足第choice需求，需要的penalty
def get_penalty(n,chioce):
    if chioce == 0:
        penalty = 0
    if chioce == 1:
        penalty = 50
    if chioce == 2:
        penalty = 50+9*n
    if chioce == 3:
        penalty = 100+9*n
    if chioce == 4:
        penalty = 200+9*n
    if chioce == 5:
        penalty = 200+18*n
    if chioce == 6:
        penalty = 300+18*n
    if chioce == 7:
        penalty = 300+36*n
    if chioce == 8:
        penalty = 400+36*n
    if chioce == 9:
        penalty = 500+(36+199)*n
    if chioce > 9:
        penalty = 500+(36+398)*n
    return penalty

In [6]:
N_DAYS = 100
N_FAMILY = 5000
MIN_OCCUPANCY = 125
MAX_OCCUPANCY = 300

In [7]:
# 计算pcost_mat,每个家庭在什么时候（day 0-99）访问时的penalty
pcost_mat = np.full(shape=(N_FAMILY,100),fill_value=999999)

In [8]:
for f in range(N_FAMILY):
    f_num = data.loc[f, 'n_people']
    pcost_mat[f, :] = get_penalty(f_num, 10)
    for choice in range(10):
        temp = data.loc[f][choice]
        penalty = get_penalty(f_num, choice)
        pcost_mat[f, temp-1] = penalty

In [11]:
pcost_mat

array([[2236, 2236, 2236, ..., 2236, 2236, 2236],
       [2236, 2236, 2236, ..., 2236, 2236, 2236],
       [1802, 1802, 1802, ..., 1802, 1802,    0],
       ...,
       [3104, 3104,  616, ..., 3104, 3104, 3104],
       [ 390, 2670, 2670, ..., 2670, 2670, 2670],
       [2236, 2236, 2236, ..., 2236, 2236, 2236]])

In [12]:
pcost_mat.shape

(5000, 100)

In [75]:
# 计算accounting penalty矩阵，从一天的参观人数，当天的参观人数
acost_mat = np.zeros(shape=(MAX_OCCUPANCY+1,MAX_OCCUPANCY+1),dtype=np.float64)
for i in range(acost_mat.shape[0]):#当天安排的人数
    for j in range(acost_mat.shape[1]):#前一天安排的人数
        diff = abs(i-j)
        acost_mat[i, j] = max(0, (i-125)/400 * i ** (0.5+diff/50.0))
acost_mat

array([[0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       ...,
       [4.16316072e+15, 3.71482922e+15, 3.31477861e+15, ...,
        7.46610759e+00, 8.36716954e+00, 9.37697794e+00],
       [4.79555148e+15, 4.27883100e+15, 3.81778713e+15, ...,
        8.43020770e+00, 7.52185316e+00, 8.43020770e+00],
       [5.52415954e+15, 4.92860244e+15, 4.39725208e+15, ...,
        9.51970597e+00, 8.49339085e+00, 7.57772228e+00]])

In [16]:
acost_mat.shape

(300, 300)

In [15]:
FAMILY_SIZE = data['n_people'].values
FAMILY_SIZE

array([4, 4, 3, ..., 6, 5, 4])

In [17]:
DESIRED = data.values[:,:-1] -1 #DESIRED代表每个家b庭的CHOICE-1
DESIRED

array([[51, 37, 11, ..., 75,  9, 27],
       [25,  3, 81, ...,  5, 65, 60],
       [99, 53, 24, ..., 88, 79, 32],
       ...,
       [31, 65, 53, ..., 80,  2,  6],
       [66, 91,  3, ..., 11, 25, 69],
       [12, 10, 24, ..., 38, 17, 46]])

In [18]:
DESIRED.shape

(5000, 10)

In [20]:
pip install ortools

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 13.7 MB 18 kB/s eta 0:00:014    |███████████████████▋            | 8.4 MB 18 kB/s eta 0:04:47
Note: you may need to restart the kernel to use updated packages.


In [21]:
from ortools.linear_solver import pywraplp

In [30]:
# 线性规划
def solve_LP():
    solver= pywraplp.Solver('AssignmentProblem', pywraplp.Solver.GLOP_LINEAR_PROGRAMMING)
    x = {} #family_id在第j天是否参观
    #每一天有哪些家庭
    candidates = [[] for x in range(N_DAYS)]
    
    for i in range(N_FAMILY):# family_id
        for j in DESIRED[i, :]:
            candidates[j].append(i)
            x[i, j] = solver.BoolVar('x[%i,%i]' %(i,j))
    
    #每天参观的人数
    daily_occupancy = [solver.Sum([x[i,j] * FAMILY_SIZE[i] for i in candidates[j]]) \
                      for j in range(N_DAYS)]
    #每个家庭在10个choice出现的情况
    family_presence = [solver.Sum(x[i, j] for j in DESIRED[i,:]) \
                       for i in range(N_FAMILY)]
    #定义目标函数
    preference_cost = solver.Sum([pcost_mat[i,j] * x[i,j] for i in range(N_FAMILY)\
                                  for j in DESIRED[i,:]])
    #目标函数最小
    solver.Minimize(preference_cost)
    #人为增加约束条件
    for j in range(N_DAYS-1):
        solver.Add(daily_occupancy[j]-daily_occupancy[j+1]<=25)
        solver.Add(daily_occupancy[j+1]-daily_occupancy[j]<=25)
    
    for i in range(N_FAMILY):
        solver.Add(family_presence[i]==1)
    
    for j in range(N_DAYS):
        solver.Add(daily_occupancy[j]>=MIN_OCCUPANCY)
        solver.Add(daily_occupancy[j]<=MAX_OCCUPANCY)
        
    result = solver.Solve()
    temp = [(i,j,x[i,j].solution_value()) for i in range(N_FAMILY) for j in DESIRED[i,:] if x[i,j].solution_value()>0]
    #print(temp)
    df = pd.DataFrame(temp, columns=['family_id','day','result'])
    return df  

In [31]:
%%time
result=solve_LP()

CPU times: user 21.9 s, sys: 0 ns, total: 21.9 s
Wall time: 25.4 s


In [32]:
result

,family_id,day,result
0,0,51,1.0
1,1,25,1.0
2,2,99,1.0
3,3,1,1.0
4,4,52,1.0
...,...,...,...
5075,4995,15,1.0
5076,4996,87,1.0
5077,4997,31,1.0
5078,4998,91,1.0


In [33]:
THRS = 0.999
assign_df = result[result.result > THRS]
assign_df

,family_id,day,result
0,0,51,1.0
1,1,25,1.0
2,2,99,1.0
3,3,1,1.0
4,4,52,1.0
...,...,...,...
5075,4995,15,1.0
5076,4996,87,1.0
5077,4997,31,1.0
5078,4998,91,1.0


In [34]:
# 没有安排的，不为0和1
unassign_df = result[(result.result < THRS) & (result.result > 1-THRS)]
unassign_df

,family_id,day,result
60,59,38,0.25
61,59,14,0.75
242,240,32,0.75
243,240,56,0.25
265,262,31,0.50
...,...,...,...
4990,4912,8,0.40
4992,4914,38,0.60
4993,4914,43,0.40
5040,4961,53,0.75


In [36]:
# 计算每天访问的人数
assign_df['family_size'] = FAMILY_SIZE[assign_df.family_id]
assign_df

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,family_id,day,result,family_size
0,0,51,1.0,4
1,1,25,1.0,4
2,2,99,1.0,3
3,3,1,1.0,2
4,4,52,1.0,4
...,...,...,...,...
5075,4995,15,1.0,4
5076,4996,87,1.0,2
5077,4997,31,1.0,6
5078,4998,91,1.0,5


In [39]:
#按照day的方式进行聚合
occupancy = assign_df.groupby('day').family_size.sum().values

In [40]:
occupancy

array([285, 271, 294, 293, 263, 242, 223, 247, 273, 297, 288, 292, 275,
       250, 245, 272, 292, 292, 271, 248, 223, 244, 264, 291, 292, 296,
       273, 249, 234, 251, 278, 283, 252, 235, 205, 184, 202, 233, 253,
       226, 210, 183, 204, 229, 247, 281, 256, 223, 204, 198, 222, 248,
       256, 223, 208, 185, 173, 196, 219, 198, 174, 141, 124, 121, 149,
       170, 178, 160, 136, 123, 125, 135, 158, 185, 158, 134, 121, 125,
       142, 167, 186, 167, 138, 120, 128, 155, 174, 203, 177, 152, 130,
       122, 130, 158, 183, 158, 128, 125, 122, 124])

In [42]:
min_occupancy = np.array([max(0,MIN_OCCUPANCY - x) for x in occupancy])

In [43]:
min_occupancy

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 4, 0, 0,
       0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 0, 0, 5, 0, 0, 0, 0,
       0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 3, 1])

In [45]:
max_occupancy = np.array([MAX_OCCUPANCY - x for x in occupancy])

In [46]:
max_occupancy

array([ 15,  29,   6,   7,  37,  58,  77,  53,  27,   3,  12,   8,  25,
        50,  55,  28,   8,   8,  29,  52,  77,  56,  36,   9,   8,   4,
        27,  51,  66,  49,  22,  17,  48,  65,  95, 116,  98,  67,  47,
        74,  90, 117,  96,  71,  53,  19,  44,  77,  96, 102,  78,  52,
        44,  77,  92, 115, 127, 104,  81, 102, 126, 159, 176, 179, 151,
       130, 122, 140, 164, 177, 175, 165, 142, 115, 142, 166, 179, 175,
       158, 133, 114, 133, 162, 180, 172, 145, 126,  97, 123, 148, 170,
       178, 170, 142, 117, 142, 172, 175, 178, 176])

In [52]:
def solveIP(families,min_occupancy,max_occupancy):
    solver = pywraplp.Solver('AssignmentProblem', pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)
    # 需要安排的家庭
    n_families = len(families)
    x = {} #family_id在第j天是否参观
    #每一天有哪些家庭
    candidates = [[] for x in range(N_DAYS)]
    
    for i in families:# family_id
        for j in DESIRED[i, :]:
            candidates[j].append(i)
            x[i, j] = solver.BoolVar('x[%i,%i]' %(i,j))
    
    #每天参观的人数
    daily_occupancy = [solver.Sum([x[i,j] * FAMILY_SIZE[i] for i in candidates[j]]) \
                      for j in range(N_DAYS)]
    #每个家庭在10个choice出现的情况
    family_presence = [solver.Sum(x[i, j] for j in DESIRED[i,:]) \
                       for i in families]
    #定义目标函数
    preference_cost = solver.Sum([pcost_mat[i,j] * x[i,j] for i in families\
                                  for j in DESIRED[i,:]])
    #目标函数最小
    solver.Minimize(preference_cost)
    
    for i in range(n_families):
        solver.Add(family_presence[i]==1)
    
    for j in range(N_DAYS):
        solver.Add(daily_occupancy[j]>=min_occupancy[j])
        solver.Add(daily_occupancy[j]<=max_occupancy[j])
        
    result = solver.Solve()
    temp = [(i,j) for i in families for j in DESIRED[i,:] if x[i,j].solution_value()>0]
    df = pd.DataFrame(temp, columns=['family_id','day'])
    return df  

In [53]:
unassign = unassign_df.family_id.unique()
#unassign
result = solveIP(unassign,min_occupancy,max_occupancy)

In [54]:
result

,family_id,day
0,59,38
1,240,32
2,262,31
3,488,39
4,500,26
...,...,...
64,4850,4
65,4886,98
66,4912,17
67,4914,38


In [58]:
df = pd.concat((assign_df[['family_id','day']],result)).sort_values('family_id')

In [59]:
df

,family_id,day
0,0,51
1,1,25
2,2,99
3,3,1
4,4,52
...,...,...
5075,4995,15
5076,4996,87
5077,4997,31
5078,4998,91


In [76]:
def pcost(prediction):
    daily_occupancy = np.zeros(N_DAYS+1,dtype=np.int64)
    penalty = 0
    for (i,p) in enumerate(prediction):
        n = FAMILY_SIZE[i]
        penalty += pcost_mat[i,p]
        daily_occupancy[p] += n
    return penalty,daily_occupancy

In [77]:
def acost(daily_occupancy):
    accounting_cost = 0
    num_out_of_range = 0
    for day in range(N_DAYS):
        n_pl = daily_occupancy[day+1]
        n = daily_occupancy[day]
        num_out_of_range += (n>MAX_OCCUPANCY) or (n<MIN_OCCUPANCY)
        accounting_cost += acost_mat[n,n_pl]
    return accounting_cost,num_out_of_range

In [81]:
def cost_function(prediction):
    penalty,daily_accupancy = pcost(prediction)
    accounting_cost,num_out_of_range = acost(daily_accupancy)
    final_score = penalty + accounting_cost +accounting_cost * 99999999
    return final_score

In [82]:
prediction = df.day.values
prediction

array([51, 25, 99, ..., 31, 91, 12])

In [83]:
print(cost_function(prediction))

3682581551269.9497


In [85]:
def save_result(pred,filename):
    result = pd.DataFrame(range(N_FAMILY),columns=['family_id'])
    result['assigned_day'] = pred+1
    result.to_csv(filename,index=False)
    return result
result = save_result(prediction, 'submission_1.csv')